In [67]:
import pandas as pd
import argparse


parser = argparse.ArgumentParser()

parser.add_argument('--file1', type=argparse.FileType('r'), help='Path to input file')
parser.add_argument('--file2', type=argparse.FileType('r'), help='Path to input file')
parser.add_argument('--file3', type=argparse.FileType('r'), help='Path to input file')
parser.add_argument('--file4', type=argparse.FileType('r'), help='Path to input file')
parser.add_argument('--initialfasta', type=argparse.FileType('r'), help='Path to input fasta file')
parser.add_argument('--output', type=argparse.FileType('r'), help='Path to output file')

args = parser.parse_args()




file1 = args.file1.name
file2 = args.file2.name
file3 = args.file3.name
file4 = args.file4.name
file_fasta = args.initialfasta.name
output = args.output.name

In [84]:
file1 = '../genome_data_sets/query_proteomes/mmseqs2_seq_clusters/clusterRes_db.tsv'
file2 = '../report_files/protein_structure_pLDDT_mean.tsv'
file3 = '../../Doctorado_kinetoplastids_genome_annotation/reports_files/fasta_header_to_uniprot_used.tsv'
file4 = '../mandatory_files/Ortholog_group_to_geneID.tsv'
file_fasta = '../genome_data_sets/query_proteomes/fasta_files/TriTrypDB-63_All_species_clean.fa'
output_info = '../report_files/ortholog_groups_structure_stats.tsv'
output = '../report_files/ortholog_groups_x_sequence_clustering_x_UNIPROT.tsv'

In [ ]:
df_clusters=pd.read_csv(file1,
           sep='\t',
           header=None,
           names=['cluster_representer','cluster_member'])

#print(df_clusters.shape)

(677726, 2)


In [70]:
df_pLDDT = pd.read_csv(file2, 
                       sep='\t', 
                       header=None, 
                       names = ['uniprot', 'pLDDT_mean'])

#print(df_pLDDT.shape)

In [71]:
df_geneId_uniprot = pd.read_csv(file3, 
                       sep='\t', 
                       names = ['Gene ID', 'uniprot'])

#print(df_geneId_uniprot.shape)

In [72]:
df_orthologG = pd.read_csv(file4, 
                       sep='\t', 
                       names = ['Ortholog_Group', 'Gene ID'])

#print(df_orthologG.shape)
#print(df_orthologG.Ortholog_Group.nunique())

In [73]:
# uno los DF para agregar a los cluster el codigo uniprot y con eso agregar el valor promedio de pLDDT
df_merged = (df_clusters
 .merge(df_geneId_uniprot, left_on='cluster_member', right_on='Gene ID', how='left')
 .merge(df_pLDDT, left_on='uniprot', right_on='uniprot', how='left')
)
#print(df_merged.shape)

(677726, 5)


In [74]:
#en cada cluster me quedo con la mejor estructura para luego agregar esta al df de cluster y asignar estructura a proteinas no modeladas.
#esto es importante porque en los cluster de homologos hay algunos de estos genes que no estan dentro no se porque pero si hago esta reduccion al menos puedo asignar una estructura al cluster.
#si no hago esto me quedo con un total de 30915 grupos de orthologos con estructura asignada. Yo asumo que debe ser porque se modelo un paralogo o algo asi y por eso no estan en el grupo.

df_merged_best_PS = (df_merged
                     .sort_values('pLDDT_mean', ascending=False)
                     .drop_duplicates(subset='cluster_representer',keep='first'))

In [75]:
#objeto para mapear al df de cluster solo con la informacion necesaria
to_map_df_merged_best_PS = df_merged_best_PS[['cluster_representer', 'uniprot', 'pLDDT_mean']]

In [76]:
#asignando estructura de proteina de alguna modelada con 90 de identidad de secuencia
df_clusters = df_clusters.merge(to_map_df_merged_best_PS, left_on='cluster_representer', right_on='cluster_representer', how='left')

In [77]:
df_orthologG_plus_structure = df_orthologG.merge(df_clusters, left_on='Gene ID', right_on='cluster_member')
df_orthologG_plus_structure = df_orthologG_plus_structure.drop_duplicates(keep='first')

In [79]:
print('Total number of otholog groups:', df_orthologG.Ortholog_Group.nunique())
print('The number of otholog groups with protein structure assigned are:', df_orthologG_plus_structure.dropna(subset=['uniprot']).Ortholog_Group.nunique())
print('Fraction:',  round(df_orthologG_plus_structure.dropna(subset=['uniprot']).Ortholog_Group.nunique() / df_orthologG.Ortholog_Group.nunique(), 3) )

Total number of otholog groups: 53705
The number of otholog groups with protein structure assigned are: 30083
Fraction: 0.56


In [92]:
#csv output info
#(df_orthologG_plus_structure
# .groupby('Ortholog_Group')['pLDDT_mean']
# .agg(['mean', 'median', 'min', 'max', 'std', 'skew'])
# .to_csv(output_info, sep='\t')
#)

In [80]:

#csv output
df_orthologG_plus_structure.sort_values(['Ortholog_Group','cluster_representer']).to_csv(output, sep='\t',index=False)

# Creating fasta file for no modelated proteins 

In [81]:
from Bio import SeqIO

In [82]:
df_orthologG_plus_structure[df_orthologG_plus_structure.uniprot.isna()]['Ortholog_Group'].nunique()

36756

In [83]:
df_orthologG_plus_structure.drop_duplicates(['Ortholog_Group', 'cluster_representer'])

,Ortholog_Group,Gene ID,cluster_representer,cluster_member,uniprot,pLDDT_mean
0,OG6_158218,TRSC58_00001,TRSC58_00001,TRSC58_00001,A0A422P0W0,96.0
1,OG6_100831,TRSC58_00002,TRSC58_00002,TRSC58_00002,A0A061JBD6,56.0
2,OG6_102428,TRSC58_00003,TRSC58_00003,TRSC58_00003,A0A061J9W1,83.0
3,OG6_173740,TRSC58_00004,TRSC58_00004,TRSC58_00004,A0A422P470,78.0
4,OG6_101177,TRSC58_00005,TRSC58_00005,TRSC58_00005,A0A422P430,73.0
...,...,...,...,...,...,...
679267,OG6r16_512199,ECC02_013890,ECC02_013890,ECC02_013890,A0A7J6XG27,50.0
679269,OG6r16_512030,ECC02_013892,ECC02_013892,ECC02_013892,A0A7J6XG20,58.0
679270,OG6_100177,ECC02_013893,ECC02_013893,ECC02_013893,A0A2V2X5C9,53.0
679277,OG6r16_116704,ECC02_013901,ECC02_013901,ECC02_013901,A0A7J6XI59,31.0
